## CNN Text Classification with Keras and Theano

In [1]:
import numpy as np
from collections import defaultdict
import _pickle as cPickle # python3 shit
import re, sys, os, csv, keras
from many_stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from itertools import chain
from nltk.classify import NaiveBayesClassifier, accuracy
from keras import regularizers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model
print("Using Keras version",keras.__version__)

Using TensorFlow backend.


Using Keras version 2.1.4


Read from dataset (`csv` file)

In [2]:
texts, labels = [], []
print("Reading from csv file...")
with open('data.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        texts.append(row[0])
        labels.append(row[1])
print("Done!")

Reading from csv file...
Done!


In [3]:
MAX_NB_WORDS = 40000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2

In [4]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

Found 30542 unique tokens.


In [5]:
labels = to_categorical(np.asarray(labels)) # convert to one-hot encoding vectors
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (40000, 1000)
Shape of label tensor: (40000, 4)


In [6]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

In [7]:
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Number of entries in each category:')
print("Training:\n",y_train.sum(axis=0))
print("Validation:\n",y_val.sum(axis=0))

Number of entries in each category:
Training:
 [ 7628. 10906. 12311.  1155.]
Validation:
 [2016. 2718. 2988.  278.]


### Preparing the Embedding layer
Compute an index mapping words to known embeddings, by parsing the data dump of pre-trained embeddings: [GloVe](https://nlp.stanford.edu/projects/glove/) vectors from Stanford NLP. For new words, a "randomised vector" will be created.

In [8]:
EMBEDDING_DIM = 200
GLOVE_DIR = "dataset/glove/glove.twitter.27B.200d.txt"
embeddings_index = {}
f = open(GLOVE_DIR)
print("Loading GloVe from:",GLOVE_DIR,"...",end="")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print("Done.\nProceeding with Embedding Matrix...")
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
print("Completed!")

Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
Proceeding with Embedding Matrix...
Completed!


After computing our embedding matrix, load this embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [9]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### Deeper 1D CNN
[Reference](https://github.com/richliao/textClassifier), [Source](https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html) and [Notes](http://www.wildml.com/2015/11/understanding-convolutional-neural-networks-for-nlp/)

Deeper Convolutional neural network: In [CNN for Sentence Classification](http://www.aclweb.org/anthology/D14-1181) (Yoon Kim, 2014), multiple filters have been applied. This can be implemented using Keras `Merge` Layer.

In [10]:
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(nb_filter=128,filter_length=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)

l_merge = Merge(mode='concat', concat_axis=1)(convs)
l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(4, activation='softmax')(l_dense)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=3)`
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=4)`
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=5)`
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  if sys.path[0] == '':


In [11]:
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])
tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=50, write_grads=True , write_graph=True)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 200)    6108600     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 998, 128)     76928       embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 997, 128)     102528      embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_3 (

In [12]:
print("Training Progress:")
model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=20, batch_size=50,
          callbacks=[tbCallBack])

Training Progress:
Train on 32000 samples, validate on 8000 samples
Epoch 1/20
32000/32000 [==============================] - 441s 14ms/step - loss: 1.1387 - acc: 0.4689 - val_loss: 1.0725 - val_acc: 0.5197
Epoch 2/20
32000/32000 [==============================] - 435s 14ms/step - loss: 1.0651 - acc: 0.5191 - val_loss: 1.0746 - val_acc: 0.5075
Epoch 3/20
32000/32000 [==============================] - 424s 13ms/step - loss: 1.0132 - acc: 0.5449 - val_loss: 1.0641 - val_acc: 0.5176
Epoch 4/20
32000/32000 [==============================] - 439s 14ms/step - loss: 0.9263 - acc: 0.5889 - val_loss: 1.1343 - val_acc: 0.5039
Epoch 5/20
  600/32000 [..............................] - ETA: 6:21 - loss: 0.8388 - acc: 0.6033

KeyboardInterrupt: 

### TODO

 - tune hyper-parameters
 - improve preprocessing
 - use drop-out